In [1]:
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

# Get data from .json

In [2]:
import json

file_path = "./data/hotel.json"

with open(file_path, 'r', encoding='utf-8') as file:
    hotels_list = json.load(file)

print(len(hotels_list))

20


In [3]:
PROMPT_TEMPLATE = """
You are a data conversion assistant. Your task is to transform each Q&A pair into 6 diverse multi-turn conversations in ShareGPT fine-tuning format.

Each input Q&A pair relates to a specific hotel. Your output must help the assistant learn to respond naturally and accurately for that **specific hotel**.

For each input Q&A pair, generate 6 JSON objects. Each object represents a distinct conversation with:
- 4 user questions and 4 assistant answers (8 messages in total).
- The **first user question** in each conversation must explicitly mention the hotel name (e.g., “Renaissance Riverside”, “Khách sạn Mường Thanh”) in a natural and varied way.
- Questions should ask the same core idea in different ways, using varied tone, length, and style (casual, formal, brief, detailed, etc.).
- **Only one conversation** (out of the six) should include a general question about the hotel’s overall information (e.g., “Cho tôi thông tin khách sạn Renaissance Riverside”, “Nói cho tôi biết về khách sạn Mường Thanh”) and the answer is full description of the hotel.
- **Only one different conversation** (out of the six) include a question asking specifically about the hotel’s location or address.

- Assistant replies should convey the same meaning (i.e., the original answer), but phrased to match the tone and wording of each question.

Each JSON object must follow the ShareGPT JSONL schema:
- Must have a top-level field: `conversations`.
- `conversations` is a list of 8 alternating messages.
- Messages alternate between `{{"from": "human", "value": ...}}` and `{{"from": "gpt", "value": ...}}`.

Output: a list of 6 JSON objects (no explanation, no extra text).

Here is hotel's information:
{hotel_information}
"""

## Send to LLM

In [4]:
from openai import OpenAI 

CLIENT = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)


In [ ]:
from typing import Dict, List

def call_openrouter_api(prompt: str) -> str:
    try:
        completion = CLIENT.chat.completions.create(
            model="qwen/qwen3-235b-a22b-2507:free",
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )

        return completion.choices[0].message.content
    
    except Exception as e:
        print(f"API called failed: {e}")
        return None

def extract_conversations(api_response: str) -> List[Dict]:
    """Get conversations in JSON type"""
    if not api_response:
        return []
    
    try:
        conversations_data = json.loads(api_response)
        return conversations_data
        # return conversations_data.get('conversations', [])
    except json.JSONDecodeError as e:
        print(f"Failed to parse API response: {e}")
        return []

def save_to_file(data: List[Dict], filename: str):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write("[")

        for entry in data:
            file.write(json.dumps(entry, ensure_ascii=False) + ",\n")

        file.write("]")
        
    print(f'Saved {len(data)} entries to {filename}')

## Loop through each hotel JSON object

In [6]:
import time

In [7]:
list_conversations = []

for index, hotel in enumerate(hotels_list):
    prompt = PROMPT_TEMPLATE.format(hotel_information=str(hotel))
    response = call_openrouter_api(prompt)

    hotel_conversations = extract_conversations(response)
    print(f"Processed succesfully hotel: {index} with {len(hotel_conversations)} conversations")

    list_conversations += hotel_conversations

    if (index + 1) % 10 == 0 and (index + 1) < len(hotels_list):
        print("Waiting 3 minutes before continuing...")
        time.sleep(180)  # 180 giây = 3 phút

Processed succesfully hotel: 0 with 6 conversations
Processed succesfully hotel: 1 with 6 conversations
Processed succesfully hotel: 2 with 6 conversations
Processed succesfully hotel: 3 with 6 conversations
Processed succesfully hotel: 4 with 6 conversations
Processed succesfully hotel: 5 with 6 conversations
Processed succesfully hotel: 6 with 6 conversations
Processed succesfully hotel: 7 with 6 conversations
Processed succesfully hotel: 8 with 6 conversations
Processed succesfully hotel: 9 with 6 conversations
Waiting 3 minutes before continuing...
Processed succesfully hotel: 10 with 6 conversations
Processed succesfully hotel: 11 with 6 conversations
Processed succesfully hotel: 12 with 6 conversations
Processed succesfully hotel: 13 with 6 conversations
Failed to parse API response: Expecting ',' delimiter: line 1 column 4162 (char 4161)
Processed succesfully hotel: 14 with 0 conversations
Processed succesfully hotel: 15 with 6 conversations
Processed succesfully hotel: 16 with 

In [8]:
save_to_file(list_conversations, 'test.json')

Saved 114 entries to test.json


## Push to Hugging Face

In [9]:
from huggingface_hub import login

login(token=HF_TOKEN)

d:\Tyan\Quickom\Simple-Fine-Tune-Llama\finetune_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [14]:
from datasets import Dataset
import json

with open("test.json", "r", encoding="utf-8") as file:
    data = json.load(file)

dataset = Dataset.from_list(data)

In [15]:
from huggingface_hub import HfApi

dataset_name = "tyanfarm/hotel_conversations_118"
api = HfApi()
api.create_repo(repo_id=dataset_name, repo_type="dataset", exist_ok=True)

RepoUrl('https://huggingface.co/datasets/tyanfarm/hotel_conversations_118', endpoint='https://huggingface.co', repo_type='dataset', repo_id='tyanfarm/hotel_conversations_118')

In [16]:
dataset.push_to_hub(dataset_name)

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.73s/ shards]
d:\Tyan\Quickom\Simple-Fine-Tune-Llama\finetune_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Tyan\.cache\huggingface\hub\datasets--tyanfarm--hotel_conversations_118. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/tyanfarm/hotel_conversations_118/commit/c27af2666d04309771c90a4313310877808b3382', commit_message='Upload dataset', commit_description='', oid='c27af2666d04309771c90a4313310877808b3382', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/tyanfarm/hotel_conversations_118', endpoint='https://huggingface.co', repo_type='dataset', repo_id='tyanfarm/hotel_conversations_118'), pr_revision=None, pr_num=None)